# Введение в Data Science
## Эпизод 6: Исчезновение учителя


Авторы теоретического материала - исследователь СколТеха Сергей Королев и программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

Основное отличие методов обучения без учителя от привычных классификаций и регрессий машинного обучения в том, что разметки для данных в этом случае нет. От этого образуются сразу несколько особенностей — во-первых это возможность использования несопоставимо больших объёмов данных, поскольку их не нужно будет размечать руками для обучения, а во-вторых это неясность измерения качества методов, из-за отсутствия таких же прямолинейных и интуитивно понятных метрик, как в задачах обучения с учителем.

## Предобработка данных
Перед тем, как мы начнем развлекаться, пока учителя нет, давайте, как обычно, поработаем с данными и приведем их к какому-то адекватному виду.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Итак, долгожданные данные по студентам, где мы объединили две группы - БМБ 177 и БМБ 172, если вы еще не знакомы, то начнем знакомиться заочно, а в конце еще и поймем, кто из вас друг на друга похож ;)

In [ ]:
data = pd.read_excel('data/dataset.xlsx')
data.head()

In [ ]:
data.tail()

In [ ]:
data.shape

Наши с вами переменные:
    
- Name - попробуйте догадаться, что это
- FavouriteSeries - ваши любимые сериалы
- StarWars - эпизоды ЗВ в порядке убывания предпочтения
- MetroTime - сколько в минутах вы тратите времени на дорогу до универа на метро
- ExamScore - суммарный балл ЕГЭ
- HomeInternetSpeed - скорость интернета дома

Какие переменные непрерывные, а какие категориальные?

### Посмотрим на пропущенные значения:

In [ ]:
data.isnull().sum()

In [ ]:
data.isnull().sum()/len(data)

Похоже, пропусков не так уж и много, давайте поработаем с каждой из переменных:

## MetroTime

In [ ]:
data.MetroTime.hist();

Похоже, есть интересное значение справа

In [ ]:
data.MetroTime.max()

In [ ]:
172800.0/60/24

Возможно, кто-то действительно тратит 120 дней своей жизни на поездки на метро, но предположим, что человек все же ошибся и заменим значение на что-то более нейтральное и медианное

In [ ]:
data.MetroTime[data.MetroTime==data.MetroTime.max()] = data.MetroTime.median()

In [ ]:
data.MetroTime.hist(normed=True, bins=10)
data.MetroTime.plot.kde()
plt.xlim(0)

### Заполнение пропусков
Если студент не указал время на метро, предположим, что он им и не пользуется, так что заполним пропуски нулями, а заодно создадим переменную - пользуется ли студент метро

In [ ]:
data.MetroTime = data.MetroTime.fillna(0)

In [ ]:
data['UseMetro'] = data.MetroTime != 0

In [ ]:
data.UseMetro.value_counts()

![](https://upload.wikimedia.org/wikipedia/commons/thumb/c/ce/Star_wars2.svg/1200px-Star_wars2.svg.png)

In [ ]:
data.StarWars.hist(bins=10);

Снова предположим, что если что-то пропущено, это значит, что человек не смотрел величайшую космическую сагу и вообще как так можно жить

![](https://i.gifer.com/Bme8.gif)

In [ ]:
data.StarWars.fillna(0, inplace=True)

In [ ]:
data.StarWars[data.StarWars>0].hist(bins=10)

Далее происходит немного магии и преобразований нашей переменной с перекодированием её в строку, заменой нулей на строку из шести нулей и разбиением одного столбца на шесть отдельных. Также не забываем дропнуть из исходного датасета закодированные переменные.

In [ ]:
data.StarWars = data.StarWars.astype(int).astype(str)
data.StarWars = data.StarWars.replace('0', '000000')
data.StarWars = data.StarWars.apply(lambda row: [x for x in row])

StarWars = pd.DataFrame(data.StarWars.tolist())
StarWars.columns = ["SW_{}".format(i) for i in range(1, 7)]

data.drop(['StarWars'], axis=1, inplace=True)

In [ ]:
StarWars.head()

Вот здесь можно было бы добавить полученные датасет с эпизодами ЗВ в наш исходный датасет, но дальнейшая практика показала, что это слишком уж мощный признак, по которому сразу  студенты делятся на две группы - тех, кто смотрел ЗВ, и тех, кто не смотрел, а это не так интересно для  нашего мини-исследования, поэтому добавлять мы сейчас этот кусок не будем :3

In [ ]:
#data = pd.concat([data, StarWars.astype(int)], axis=1)

## ExamScore

![](https://i.ytimg.com/vi/inY2_sZCw20/hqdefault.jpg)

In [ ]:
data.ExamScore.hist();

In [ ]:
data.ExamScore.max()

In [ ]:
data[data.ExamScore==1337]

Впечатляющий результат! Чуров был бы доволен. Но скорее всего единичка затесалась туда нечаянно, так что уберем лишнюю тысячу баллов

In [ ]:
data.ExamScore[data.ExamScore==data.ExamScore.max()] = 337

In [ ]:
plt.figure(figsize=(7, 5))
data.ExamScore.hist(normed=True)
data.ExamScore.plot.kde();

Судя по равпределению, кто-то сдавал 3 экзамена, кто-то 4, кто-то указал всего 1, так что надо бы это дело привести к чему-то более-менее однородному, давайте сначала посчитаем, сколько человек сдавал экзаменов.

Для этого возьмем целый остаток от деления на $100$ и прибавим к нему $1$, если есть нецелый остаток от $100$. Например, если у человека 350 баллов, то первая часть нам даст $350//100 = 3$, а вторая $350%100=50$; $50 > 0 = 1 (True)$; итого $3 + 1 = 4$ экзамена

In [ ]:
data['ExamQuantity'] = data.ExamScore//100 + (data.ExamScore%100 > 0).astype(int)
data['ExamQuantity'] = data['ExamQuantity'].fillna(0).astype(int)

In [ ]:
data.ExamQuantity.value_counts().plot.bar();

А теперь добавим переменную со средним баллом, потому что мы можем, и потому что это круто

In [ ]:
data['AverageScore'] = data.ExamScore.div(data.ExamQuantity)

In [ ]:
data.AverageScore.hist(bins=5, normed=True)
data.AverageScore.plot.kde()
plt.title("Распределение среднего балла ЕГЭ");

Теперь мы можем облегченно дропнуть наш изначальный ExamScore, так как у нас появились две новых переменных, его кодирующих. А заодно еще добавим переменную ScoreUnknown, чтобы отслеживать тех, кто ЕГЭ либо не сдавал, либо благополучно успел забыть свой результат

In [ ]:
data.drop(['ExamScore'], axis=1, inplace=True)

In [ ]:
data['ScoreUnknown'] = data.AverageScore.isnull()

In [ ]:
data.ScoreUnknown.value_counts()

Пропущенные значения снова заполним медианами, чтобы они нам не мешались

In [ ]:
data.AverageScore = data.AverageScore.fillna(data.AverageScore.median())

In [ ]:
data.head()

## HomeInternetSpeed

![](https://i0.wp.com/www.developermemes.com/wp-content/uploads/2015/02/Chrome-Vs-Firefox-While-IE-Eats-Glue.jpg?fit=698%2C501)

In [ ]:
data.HomeInternetSpeed.value_counts()

Закодируем слова цифрами, здесь не нужно делать OHE так как мы заведомо предполагаем, что одна категория лучше (больше) другой и можем это передать при помощи циферок

In [ ]:
data.HomeInternetSpeed = data.HomeInternetSpeed.map({'хорошо':1, "средне":0, "плохо":-1})

Снова запомним тех, про кого мы не знаем

In [ ]:
data['InternetSpeedUnknown'] = data.HomeInternetSpeed.isnull()

И заполним пропуски нулями (категория средне)

In [ ]:
data.HomeInternetSpeed = data.HomeInternetSpeed.fillna(0).astype(int)

In [ ]:
data.head()

## FavoriteSeries
Самое интересное и сложненькое

![](https://s3.amazonaws.com/dailybreak_images_prod/4c62fad1-004c-4cf5-8cb1-1b873ea4facb)

Для начала - переведем весь текст в нижний регистр

In [ ]:
data.FavoriteSeries = data.FavoriteSeries.str.lower()

Теперь заполним все пропуски кодовым словом noseries

In [ ]:
data.FavoriteSeries.fillna('noseries', inplace=True)

Теперь не особо хитрыми манимуляциями сначала разобьем каждую строку по запятой, чтобы  получить отдельные сериалы, а затем те названия сериалов, которые состоят из отдельных слова (рик и морти) склеим друг с другом через нижнее подчеркивание (рик\_и\_морти) и опять преобразуем в целую строку

In [ ]:
data.FavoriteSeries_processed = data.FavoriteSeries.str.split(',')
print(data.FavoriteSeries_processed.head())

data.FavoriteSeries_processed = data.FavoriteSeries_processed.apply(
    lambda row: [x.strip().replace(' ', '_') for x in row])
print(data.FavoriteSeries_processed.head())

data.FavoriteSeries_processed = data.FavoriteSeries_processed.apply(lambda row: ",".join(row))
print(data.FavoriteSeries_processed.head())

А теперь мы будем считать, кто чего смотрел при помощи CountVectorizer - как он устроен посмотрим на доске

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [ ]:
FavoriteSeries_processed = vectorizer.fit_transform(data.FavoriteSeries_processed)

In [ ]:
FavoriteSeries_processed

Не так уж много уникальных сериалов - всего 95, так что можно из разреженной матрицы перейти к датафрему

In [ ]:
FavoriteSeries_processed = pd.DataFrame(FavoriteSeries_processed.toarray())
FavoriteSeries_processed.columns = vectorizer.get_feature_names()

In [ ]:
FavoriteSeries_processed.head()

In [ ]:
FavoriteSeries_processed.sum().sort_values(ascending=False).head(10).plot.bar();

Выделим отдельно топ-10 сериалов, которые смотрит большинство

In [ ]:
top_ten_series = FavoriteSeries_processed.sum().sort_values(ascending=False).head(10).index

In [ ]:
plt.figure(figsize = (12, 10))
sns.heatmap(FavoriteSeries_processed[top_ten_series].corr('spearman'));

Добавим этим топ-10 к датасету и выкинем изначальную переменную, которую мы закодировали

In [ ]:
data = pd.concat([data, FavoriteSeries_processed[top_ten_series]], axis=1)

In [ ]:
data['TotalSeriesWatched'] = FavoriteSeries_processed.sum(1)

In [ ]:
data.head()

## NameLength
Напоследок, добавим длину имени в качестве признака и на этом успокоимся

In [ ]:
data['NameLength'] = data.Name.apply(len)
data.set_index(data.Name, inplace=True)

In [ ]:
data.drop(['Name', 'FavoriteSeries'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize = (12, 10))
sns.heatmap(data.corr('spearman'));

In [ ]:
data.columns

![](https://media.collegetimes.com/uploads/2015/03/its-done.gif)

## Кластеризация

Интуитивная постановка задачи кластеризации довольно проста и представляет из себя наше желание сказать: "Вот тут у меня насыпаны точки. Я вижу, что они сваливаются в какие-то кучки вместе. Было бы круто иметь возможность эти точки относить к кучкам и в случае появления новой точки на плоскости говорить, в какую кучку она падает." Из такой постановки видно, что пространства для фантазии получается много, и от этого возникает соответствующее множество алгоритмов решения этой задачи. Перечисленные алгоритмы ни в коем случае не описывают данное множество полностью, но являются примерами самых популярных методов решения задачи кластеризации.


<figure><img align="center" src="https://habrastorage.org/getpro/habr/post_images/8b9/ae5/586/8b9ae55861f22a2809e8b3a00ef815ad.png"><figcaption>Примеры работы алгоритмов кластеризации из документации пакета scikit-learn</figcaption></figure>

### K-means

Алгоритм К-средних, наверное, самый популярный и простой алгоритм кластеризации и очень легко представляется в виде простого псевдокода:
1. Выбрать количество кластеров $k$, которое нам кажется оптимальным для наших данных.
2. Высыпать случайным образом в пространство наших данных $k$ точек (центроидов).
3. Для каждой точки нашего набора данных посчитать, к какому центроиду она ближе.
4. Переместить каждый центроид в центр выборки, которую мы отнесли к этому центроиду.
5. Повторять последние два шага фиксированное число раз, либо до тех пор пока центроиды не "сойдутся" (обычно это значит, что их смещение относительно предыдущего положения не превышает какого-то заранее заданного небольшого значения).

Начнем мы с того, что отшкалируем наши непрерывные переменные, чтобы все они были в одном масштабе

In [ ]:
from sklearn.preprocessing import StandardScaler

# преобразуем все признаки в числовые
X = data.copy()
scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[['MetroTime', 'AverageScore', 'NameLength', 'TotalSeriesWatched']] =\
scaler.fit_transform(X[['MetroTime', 'AverageScore', 'NameLength', 'TotalSeriesWatched']])

Для разнообразия можем снова посомтреть, как наши многомерные данные выглядят на двумерной плоскости

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
%%time
tsne = TSNE(random_state=17)
tsne_representation = tsne.fit_transform(X_scaled)

In [ ]:
plt.scatter(tsne_representation[:, 0], tsne_representation[:, 1]);

Похоже, что-то разделить явно можно

## Выбор числа кластеров для kMeans

В отличие от задачи классификации или регресии, в случае кластеризации сложнее выбрать критерий, с помощью которого было бы просто представить задачу кластеризации как задачу оптимизации.
В случае kMeans распространен вот такой критерий – сумма квадратов расстояний от точек до центроидов кластеров, к которым они относятся.
$$ J(C) = \sum_{k=1}^K\sum_{i~\in~C_k} ||x_i - \mu_k|| \rightarrow \min\limits_C,$$

здесь $C$ – множество кластеров мощности $K$, $\mu_k$ – центроид кластера $C_k$.

Понятно, что здравый смысл в этом есть: мы хотим, чтобы точки распологались кучно возле центров своих кластеров. Но вот незадача: минимум такого фнукционала будет достигаться тогда, когда кластеров столько же, сколько и точек (то есть каждая точка – это кластер из одного элемента).
Для решения этого вопроса (выбора числа кластеров) часто пользуются такой эвристикой: выбирают то число кластеров, начиная с которого описанный функционал $ J(C) $ падает "уже не так быстро". 

In [ ]:
from sklearn.cluster import KMeans       # сама модель
from sklearn import metrics              # куда ж мы без метрик
from scipy.spatial.distance import cdist # функция для рассчета расстояний между парами точек

# будем искать оптимальное k
inertia = []
for k in range(1, 8):
    kmeans = KMeans(n_clusters=k, random_state=1).fit(X_scaled)
    inertia.append(np.sqrt(kmeans.inertia_))

In [ ]:
plt.plot(range(1, 8), inertia, marker='s');
plt.xlabel('$k$')
plt.ylabel('$J(C_k)$');

Кажется, после двух кластеров наш функционал ошибки стал уменьшаться не так быстро

In [ ]:
kmeanModel = KMeans(n_clusters=2)
kmeanModel.fit(X_scaled)

Из модели мы можем вытаскивать также и лэйблы классов

In [ ]:
kmeanModel.labels_

In [ ]:
X['clusterLabels'] = kmeanModel.labels_
X_scaled['clusterLabels'] = kmeanModel.labels_
X.clusterLabels.value_counts()

Снова можем посмотреть на нашу двумерную репрезентацию

In [ ]:
labels = list(kmeanModel.labels_)

plt.scatter(tsne_representation[:, 0], tsne_representation[:, 1], 
            c=X['clusterLabels'].map({0: 'blue', 1: 'orange'}));

Наконец, можем посмотреть, чем, в среднем, отличаются две группы

In [ ]:
X_scaled.groupby('clusterLabels').mean().T.plot.barh(figsize=(8, 10));

In [ ]:
X_scaled[['clusterLabels']]

# Агломеративная или иерархическая кластеризация

Наверное самый простой и понятный алгоритм кластеризации без фиксированного числа кластеров — агломеративная кластеризация. Интуиция у алгоритма очень простая: 
1. Начинаем с того, что высыпаем на каждую точку свой кластер
2. Сортируем попарные расстояния между центрами кластеров по возрастанию
3. Берём пару ближайших кластеров, склеиваем их в один и пересчитываем центр кластера
4. Повторяем п. 2 и 3 до тех пор, пока все данные не склеятся в один кластер

По итогам выполнения такого алгоритма можно построить замечательное дерево склеивания кластеров и глядя на него определить, на каком этапе нам было бы оптимальнее всего остановить алгоритм. Либо воспользоваться тем же правилом локтя, что и в k-means.

In [ ]:
from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist

distance_mat = pdist(X_scaled) # pdist посчитает нам верхний треугольник матрицы попарных расстояний

Z = hierarchy.linkage(distance_mat, 'single') # linkage — реализация агломеративного алгоритма
plt.figure(figsize=(10, 20))
dn = hierarchy.dendrogram(Z, color_threshold=2.1, labels=X.index,leaf_font_size=12., orientation='left')

![](https://i.gifer.com/7CXm.gif)

![](http://i0.kym-cdn.com/photos/images/newsfeed/000/707/322/fac.gif)